In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context('poster')
sns.set_style('ticks')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'cmr10'
plt.rcParams["mathtext.fontset"] = 'cm'
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units

These are quicklook plots to evaluate the representation of clouds in the analyses/reanalyses datasets.

---

In [ ]:
h = 30. # surface elevation of ENA site in m
lat = 39.0916 # deg N
lon = -28.0257 # deg E
dx = 1.5 # deg, half-width of averaging boxes for omega etc.

In [ ]:
obs_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/obs'
forc_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/forcing'

In [ ]:
varanal = xr.open_dataset(f'{obs_dir}/ena180varanaecmwfC1.c1.20170601.000000.cdf')
va = varanal.assign_coords(time=varanal.time_offset)
time_va = np.arange(0, 23.9, 3)

In [ ]:
# Code from ECWMF for calculating geopotential on model levels

# # compute moist temperature
# t_level = t_level * (1. + 0.609133 * q_level)

# # compute the pressures (on half-levels)
# ph_lev, ph_levplusone = get_ph_levs(values, lev)

# if lev == 1:
#     dlog_p = np.log(ph_levplusone / 0.1)
#     alpha = np.log(2)
# else:
#     dlog_p = np.log(ph_levplusone / ph_lev)
#     alpha = 1. - ((ph_lev / (ph_levplusone - ph_lev)) * dlog_p)

# t_level = t_level * R_D

# # z_f is the geopotential of this full level
# # integrate from previous (lower) half-level z_h to the
# # full level
# z_f = z_h + (t_level * alpha)

# # z_h is the geopotential of 'half-levels'
# # integrate z_h to next half level
# z_h = z_h + (t_level * dlog_p)

# return z_h, z_f

In [ ]:
def calc_tp_era5(data):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    v = np.genfromtxt('era5_table.csv', delimiter=',', skip_header=1, missing_values='-')
    a = v[:,1]
    b = v[:,2]
    t = data.t
    lnsp = data.lnsp.values[:,0,:,:]
    nt, nz, ny, nx = t.shape
    pi = np.zeros((nt, nz+1, ny, nx))
    pi[:] = np.exp(lnsp[:,np.newaxis,:,:])
    pi = a[np.newaxis,:,np.newaxis, np.newaxis]+pi*b[np.newaxis,:,np.newaxis, np.newaxis]
    p = (pi[:,1:,:,:] + pi[:,:-1,:,:])*0.5
    tpv = metpy.calc.potential_temperature(p*units.pascal, t)
    tp = t.copy(deep=True, data=tpv)
    tp.rename('Air Potential Temperature')
    return tp

def extract_era5_profiles(data, time):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    v = np.genfromtxt('era5_table.csv', delimiter=',', skip_header=1, missing_values='-')
    a = v[:,1]
    b = v[:,2]
    t = data.t.loc[time].values
    qv = data.q.loc[time].values
    lnsp = data.lnsp.loc[time].values[0]
    sgp = data.z.loc[time].values[0]
    nz = t.shape[0]
    pi = np.zeros(nz+1)
    pi[:] = np.exp(lnsp)
    pi = a+pi*b
    p = (pi[1:] + pi[:-1])*0.5
    # Pa, only used for geopotential calculation below
    pi[0] = 0.1
    tm = t*(1.0+0.609133*qv)*units.kelvin
    dpi = pi[1:] - pi[:-1]
    dlnpi = np.log(pi[1:]/pi[:-1])
    # have not got time to derive alpha, this is just what is given in the ERA5 documentation
    alpha = 1.0 - dlnpi*pi[:-1]/dpi
    # following ERA5 doc
    alpha[0] = np.log(2.)
    dph = rd*tm*dlnpi
    ph = np.zeros(nz+1)
    ph[:-1][::-1] = np.cumsum(dph[::-1])
    ph[:] = ph[:] + sgp
    zi = ph*units('m**2/s**2')/g
    z = zi[1:] + rd*tm*alpha/g
    tp = metpy.calc.potential_temperature(p*units.pascal, t*units.kelvin)
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    return z, p*units.pascal, tp, rv

In [ ]:
def extract_merra2_profiles_ana(data, time, sgp=196.70242):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    ptop = data.ptop
    dpi = data.DELP.loc[time] 
    nz = dpi.shape[0]
    t = data.T.loc[time].metpy.quantify()
    qv = data.QV.loc[time].metpy.quantify()
    tm = t*(1.0+0.609133*qv)
    pi = np.zeros(nz+1)
    pi[1:] = np.cumsum(dpi) + ptop
    pi[0] = ptop
    p = (pi[1:] + pi[:-1])*0.5
    dlnpi = np.log(pi[1:]/pi[:-1])
    alpha = 1.0 - dlnpi*pi[:-1]/dpi
    dph = rd*tm*dlnpi
    ph = np.zeros(nz+1)
    ph[:-1][::-1] = np.cumsum(dph[::-1])
    ph[:] = ph[:] + sgp
    zi = ph*units('m**2/s**2')/g
    z = zi[1:] + rd*tm*alpha/g
    tp = metpy.calc.potential_temperature(p*units.pascal, t)
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    return z, p*units.pascal, tp, rv

def extract_merra2_profiles_asm(data, time, sgp=196.70242):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    z = data.H.loc[time].metpy.quantify()
    t = data.T.loc[time].metpy.quantify()
    qv = data.QV.loc[time].metpy.quantify()
    p = data.PL.loc[time].metpy.quantify()
    tp = metpy.calc.potential_temperature(p, t)
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    return z, p, tp, rv

In [ ]:
time_armbe = np.arange(0.5, 23.7, 1.0)
armbe = xr.open_dataset(f'{obs_dir}/enaarmbeatmC1.c1.20180101.003000.nc')
armbecld = xr.open_dataset(f'{obs_dir}/enaarmbecldradC1.c1.20180101.003000.nc')

In [ ]:
time_era5 = np.arange(0, 23.9, 1.)
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-sfc.nc')
era5_l = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-ml.nc')
era5_sfc_ena = era5_sfc.sel(longitude=lon, latitude=lat, method='nearest')
era5_l_ena = era5_l.sel(longitude=lon+360., latitude=lat, method='nearest')
era5_sfc_ena = era5_sfc_ena.sortby(era5_sfc_ena.time)

In [ ]:
fnl_sfc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_surface.nc')
fnl_sfc2 = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_heightAboveGround.nc')
fnl_cwat = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_atmosphereSingleLayer.nc')
fnl_tcc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_atmosphere.nc')
fnl_lcc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_lowCloudLayer.nc')
fnl_l = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_isobaricInhPa.nc')
fnl_sfc_ena = fnl_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_sfc2_ena = fnl_sfc2.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_tcc_ena = fnl_tcc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_lcc_ena = fnl_lcc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_cwat_ena = fnl_cwat.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_l_ena = fnl_l.sel(longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.20180207.nc4')
merra2_l_ana = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst6_3d_ana_Nv.20180207.nc4')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180207.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180207.nc4')
merra2_cld = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_rad_Nx.20180207.nc4')
merra2_l_ena = merra2_l.sel(lon=lon, lat=lat, method='nearest')
merra2_l_ena_ana = merra2_l_ana.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc_ena = merra2_sfc.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc2_ena = merra2_sfc2.sel(lon=lon, lat=lat, method='nearest')
merra2_cld_ena = merra2_cld.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
datestring='20180207'
arscl = xr.open_dataset(f'{obs_dir}/enaarsclkazr1kolliasC1/enaarsclkazr1kolliasC1.c0.{datestring}.000000.nc')

In [ ]:
datestring='2018-02-07'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
tcc = armbecld.tot_cld.loc[t1:t2]
tcc_tsi = armbecld.tot_cld_tsi.loc[t1:t2]
tcc_visst = armbecld.cld_tot.loc[t1:t2]*0.01
ax.plot(time_armbe, tcc, color='black', label='ARMBE')
ax.plot(time_armbe, tcc_tsi, color='black', linestyle='--', label='ARMBE TSI')
ax.plot(time_armbe, tcc_visst, color='black', linestyle=':', label='ARMBE GOES')
# ERA5
tcc = era5_sfc_ena.tcc.loc[t1:t2]
ax.plot(time_era5, tcc, color='black', linestyle='none', marker='+', label='ERA5')
lcc = era5_sfc_ena.lcc.loc[t1:t2]
ax.plot(time_era5, lcc, color='red', linestyle='none', marker='+', label='low cld in red')
mcc = era5_sfc_ena.mcc.loc[t1:t2]
ax.plot(time_era5, mcc, color='green', linestyle='none', marker='+', label='mid cld')
hcc = era5_sfc_ena.hcc.loc[t1:t2]
ax.plot(time_era5, hcc, color='blue', linestyle='none', marker='+', label='high cld')
# VA
tcc = va.cld_tot.loc[t1:t2]*0.01
ax.plot(time_va, tcc, color='black', linestyle='none', marker='^', label='VarAnal')
lcc = va.cld_low.loc[t1:t2]*0.01
ax.plot(time_va, lcc, color='red', linestyle='none', marker='^')
# FNL
time_fnl = np.arange(0, 23.9, 6.)
tcc = fnl_tcc_ena.tcc.loc[t1:t2]*0.01
ax.plot(time_fnl, tcc, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
lcc = fnl_lcc_ena.tcc.loc[t1:t2]*0.01
ax.plot(time_fnl, lcc, color='red', linestyle='none', marker='*', markersize=10)
# MERRA2
tcc = merra2_cld_ena.CLDTOT.loc[t1:t2]
ax.plot(time_merra2, tcc, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
lcc = merra2_cld_ena.CLDLOW.loc[t1:t2]
ax.plot(time_merra2, lcc, color='red', linestyle='none', marker='o', markersize=5)
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.ylim(0, 1.05)
plt.axhspan(90, 310, color='lightgray', alpha=0.2)
plt.ylabel('Cloud Fraction')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring='2018-02-07'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:59'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
cbh = arscl.cloud_base_best_estimate.loc[t1:t2]
time_cbh = np.arange(0, 86400, 4)/3600.
ax.plot(time_cbh, cbh, color='black')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.ylabel('Cloud Base Height (m)')
plt.title(datestring)
plt.show()

In [ ]:
datestring='2018-02-07'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
lwp = armbecld.lwp.loc[t1:t2]
ax.plot(time_armbe, lwp, color='black', label='ARMBE')
# ERA5
lwp = era5_sfc_ena.tclw.loc[t1:t2]*1.0e3
ax.plot(time_era5, lwp, color='black', linestyle='none', marker='+', label='ERA5')
# VA
lwp = va.LWP.loc[t1:t2]*1.0e4 # cm to g/m^2
ax.plot(time_va, lwp, color='black', linestyle='none', marker='^', label='VarAnal')
# FNL
time_fnl = np.arange(0, 23.9, 3.)
lwp = fnl_cwat_ena.cwat.loc[t1:t2]*1.0e3
ax.plot(time_fnl, lwp, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
# MERRA2
lwp = merra2_sfc2_ena.TQL.loc[t1:t2]*1.0e3
ax.plot(time_merra2, lwp, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
# plt.ylim(0, 1.05)
plt.ylabel(r'LWP ($g/m^2$)')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring='2018-02-07'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
swdn = armbecld.swdn.loc[t1:t2]
ax.plot(time_armbe, swdn, color='black', label='ARMBE')
cswdn = armbecld.clrswfluxdn.loc[t1:t2]
ax.plot(time_armbe, cswdn, color='red')
# # ERA5
# lwp = era5_sfc_ena.tclw.loc[t1:t2]*1.0e3
# ax.plot(time_era5, lwp, color='black', linestyle='none', marker='+', label='ERA5')
# VA
swdn = va.sw_dn_srf.loc[t1:t2]
ax.plot(time_va, swdn, color='black', linestyle='none', marker='^', label='VarAnal')
# FNL
# half of the data points are nans
time_fnl = np.arange(0, 23.9, 3.)
swdn = fnl_sfc_ena.dswrf.loc[t1:t2]
# print(swdn.values)
ax.plot(time_fnl, swdn, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
# MERRA2
swdn = merra2_cld_ena.SWGDN.loc[t1:t2]
cswdn = merra2_cld_ena.SWGDNCLR.loc[t1:t2]
ax.plot(time_merra2, swdn, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
ax.plot(time_merra2, cswdn, color='red', linestyle='none', marker='o', markersize=5, label='clearsky in red')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
# plt.ylim(0, 1.05)
plt.ylabel(r'SWDN ($W/m^2$)')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring='2018-02-07'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
lwdn = armbecld.lwdn.loc[t1:t2]
ax.plot(time_armbe, lwdn, color='black', label='ARMBE')
# # ERA5
# lwp = era5_sfc_ena.tclw.loc[t1:t2]*1.0e3
# ax.plot(time_era5, lwp, color='black', linestyle='none', marker='+', label='ERA5')
# VA
lwdn = va.lw_dn_srf.loc[t1:t2]
ax.plot(time_va, lwdn, color='black', linestyle='none', marker='^', label='VarAnal')
# FNL
time_fnl = np.arange(0, 23.9, 3.)
lwdn = fnl_sfc_ena.dlwrf.loc[t1:t2]
ax.plot(time_fnl, lwdn, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
# MERRA2
lwdn = merra2_cld_ena.LWGAB.loc[t1:t2]
ax.plot(time_merra2, lwdn, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
# plt.ylim(0, 1.05)
plt.ylabel(r'LWDN ($W/m^2$)')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring = '2018-02-07'
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# 1200 UTC
timestring = '12:00'
tstamp = f'{datestring}T{timestring}:00'
col = 'red'
# ERA5
center = 'ERA5'
marker_args = {'marker':'+'}
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, tstamp)
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
qc = era5_l_ena.clwc.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = era5_l_ena.cc.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# FNL
center = 'FNL'
marker_args = {'marker':'*'}
z = fnl_l_ena.gh.loc[tstamp]
lim = z < 5000.
zlim = np.compress(lim, z)
qc = fnl_l_ena.clwmr.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# MERRA2
center = 'MERRA2'
marker_args = {'marker':'o', 'markersize':5}
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, tstamp)
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
qc = merra2_l_ena.QL.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = merra2_l_ena.CLOUD.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cbh = np.nanmean(np.where(arscl.cloud_base_best_estimate.loc[f'{datestring}T11:30:00':f'{datestring}T12:30:00'] > 0.0, arscl.cloud_base_best_estimate.loc[f'{datestring}T11:30:00':f'{datestring}T12:30:00'], np.nan))
cbh = cbh + 30.
ax1.axhline(cbh*1.0e-3, color='red')
ax2.axhline(cbh*1.0e-3, color='red')
# 1800 UTC
timestring = '18:00'
tstamp = f'{datestring}T{timestring}:00'
col = 'blue'
# ERA5
center = 'ERA5'
marker_args = {'marker':'+'}
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, tstamp)
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
qc = era5_l_ena.clwc.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = era5_l_ena.cc.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# FNL
center = 'FNL'
marker_args = {'marker':'*'}
z = fnl_l_ena.gh.loc[tstamp]
lim = z < 5000.
zlim = np.compress(lim, z)
qc = fnl_l_ena.clwmr.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# MERRA2
center = 'MERRA2'
marker_args = {'marker':'o', 'markersize':5}
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, tstamp)
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
qc = merra2_l_ena.QL.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = merra2_l_ena.CLOUD.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cbh = np.nanmean(np.where(arscl.cloud_base_best_estimate.loc[f'{datestring}T17:30:00':f'{datestring}T18:30:00'] > 0.0, arscl.cloud_base_best_estimate.loc[f'{datestring}T17:30:00':f'{datestring}T18:30:00'], np.nan))
cbh = cbh + 30.
ax1.axhline(cbh*1.0e-3, color='blue')
ax2.axhline(cbh*1.0e-3, color='blue')
ax1.set_ylim(0, 5)
ax1.set_ylabel('Altitude (km)')
ax1.set_xlabel(r'$r_c$ (g/kg)')
ax1.set_title(datestring)
ax1.legend(fontsize=12)
ax2.set_ylim(0, 5)
ax2.set_ylabel('')
ax2.set_xlabel('Cloud Fraction')
ax2.set_title(datestring)
plt.show()

---

In [ ]:
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180215-20180217-sfc.nc')
era5_l = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180215-20180217-ml.nc')
era5_sfc_ena = era5_sfc.sel(longitude=lon, latitude=lat, method='nearest')
era5_l_ena = era5_l.sel(longitude=lon+360., latitude=lat, method='nearest')
era5_sfc_ena = era5_sfc_ena.sortby(era5_sfc_ena.time)

In [ ]:
fnl_sfc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_surface.nc')
fnl_sfc2 = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_heightAboveGround.nc')
fnl_cwat = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_atmosphereSingleLayer.nc')
fnl_tcc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_atmosphere.nc')
fnl_lcc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_lowCloudLayer.nc')
fnl_l = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_isobaricInhPa.nc')
fnl_sfc_ena = fnl_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_sfc2_ena = fnl_sfc2.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_tcc_ena = fnl_tcc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_lcc_ena = fnl_lcc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_cwat_ena = fnl_cwat.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_l_ena = fnl_l.sel(longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.20180216.nc4')
merra2_l_ana = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst6_3d_ana_Nv.20180216.nc4')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180216.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180216.nc4')
merra2_cld = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_rad_Nx.20180216.nc4')
merra2_l_ena = merra2_l.sel(lon=lon, lat=lat, method='nearest')
merra2_l_ena_ana = merra2_l_ana.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc_ena = merra2_sfc.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc2_ena = merra2_sfc2.sel(lon=lon, lat=lat, method='nearest')
merra2_cld_ena = merra2_cld.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
datestring='20180216'
arscl = xr.open_dataset(f'{obs_dir}/enaarsclkazr1kolliasC1/enaarsclkazr1kolliasC1.c0.{datestring}.000000.nc')

In [ ]:
datestring='2018-02-16'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:59'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
cbh = arscl.cloud_base_best_estimate.loc[t1:t2]
time_cbh = np.arange(0, 86400, 4)/3600.
ax.plot(time_cbh, cbh, color='black')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.ylim(0, 2500)
plt.xticks((0, 6, 12, 18))
plt.ylabel('Cloud Base Height (m)')
plt.title(datestring)
plt.show()

In [ ]:
datestring='2018-02-16'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
tcc = armbecld.tot_cld.loc[t1:t2]
tcc_tsi = armbecld.tot_cld_tsi.loc[t1:t2]
tcc_visst = armbecld.cld_tot.loc[t1:t2]*0.01
ax.plot(time_armbe, tcc, color='black', label='ARMBE')
ax.plot(time_armbe, tcc_tsi, color='black', linestyle='--', label='ARMBE TSI')
ax.plot(time_armbe, tcc_visst, color='black', linestyle=':', label='ARMBE GOES')
# ERA5
tcc = era5_sfc_ena.tcc.loc[t1:t2]
ax.plot(time_era5, tcc, color='black', linestyle='none', marker='+', label='ERA5')
lcc = era5_sfc_ena.lcc.loc[t1:t2]
ax.plot(time_era5, lcc, color='red', linestyle='none', marker='+')
# VA
tcc = va.cld_tot.loc[t1:t2]*0.01
ax.plot(time_va, tcc, color='black', linestyle='none', marker='^', label='VarAnal')
lcc = va.cld_low.loc[t1:t2]*0.01
ax.plot(time_va, lcc, color='red', linestyle='none', marker='^')
# FNL
time_fnl = np.arange(0, 23.9, 6.)
tcc = fnl_tcc_ena.tcc.loc[t1:t2]*0.01
ax.plot(time_fnl, tcc, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
lcc = fnl_lcc_ena.tcc.loc[t1:t2]*0.01
ax.plot(time_fnl, lcc, color='red', linestyle='none', marker='*', markersize=10)
# MERRA2
tcc = merra2_cld_ena.CLDTOT.loc[t1:t2]
ax.plot(time_merra2, tcc, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
lcc = merra2_cld_ena.CLDLOW.loc[t1:t2]
ax.plot(time_merra2, lcc, color='red', linestyle='none', marker='o', markersize=5, label='low cloud in red')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
plt.ylim(0, 1.05)
plt.axhspan(90, 310, color='lightgray', alpha=0.2)
plt.ylabel('Cloud Fraction')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring='2018-02-16'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
lwp = armbecld.lwp.loc[t1:t2]
ax.plot(time_armbe, lwp, color='black', label='ARMBE')
# ERA5
lwp = era5_sfc_ena.tclw.loc[t1:t2]*1.0e3
ax.plot(time_era5, lwp, color='black', linestyle='none', marker='+', label='ERA5')
# VA
lwp = va.LWP.loc[t1:t2]*1.0e4 # cm to g/m^2
ax.plot(time_va, lwp, color='black', linestyle='none', marker='^', label='VarAnal')
# FNL
time_fnl = np.arange(0, 23.9, 3.)
lwp = fnl_cwat_ena.cwat.loc[t1:t2]*1.0e3
# print(lwp.values)
ax.plot(time_fnl, lwp, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
# MERRA2
lwp = merra2_sfc2_ena.TQL.loc[t1:t2]*1.0e3
ax.plot(time_merra2, lwp, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
# plt.ylim(0, 1.05)
plt.ylabel(r'LWP ($g/m^2$)')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring='2018-02-16'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
swdn = armbecld.swdn.loc[t1:t2]
ax.plot(time_armbe, swdn, color='black', label='ARMBE')
cswdn = armbecld.clrswfluxdn.loc[t1:t2]
ax.plot(time_armbe, cswdn, color='red')
# # ERA5
# lwp = era5_sfc_ena.tclw.loc[t1:t2]*1.0e3
# ax.plot(time_era5, lwp, color='black', linestyle='none', marker='+', label='ERA5')
# VA
swdn = va.sw_dn_srf.loc[t1:t2]
ax.plot(time_va, swdn, color='black', linestyle='none', marker='^', label='VarAnal')
# FNL
time_fnl = np.arange(0, 23.9, 3.)
swdn = fnl_sfc_ena.dswrf.loc[t1:t2]
ax.plot(time_fnl, swdn, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
# MERRA2
swdn = merra2_cld_ena.SWGDN.loc[t1:t2]
cswdn = merra2_cld_ena.SWGDNCLR.loc[t1:t2]
ax.plot(time_merra2, swdn, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
ax.plot(time_merra2, cswdn, color='red', linestyle='none', marker='o', markersize=5, label='clearsky in red')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
# plt.ylim(0, 1.05)
plt.ylabel(r'SWDN ($W/m^2$)')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring='2018-02-16'
t1 = f'{datestring}T00:00:00'
t2 = f'{datestring}T23:59:00'
fig = plt.figure(figsize=(9, 6))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# ARMBE
lwdn = armbecld.lwdn.loc[t1:t2]
ax.plot(time_armbe, lwdn, color='black', label='ARMBE')
# # ERA5
# lwp = era5_sfc_ena.tclw.loc[t1:t2]*1.0e3
# ax.plot(time_era5, lwp, color='black', linestyle='none', marker='+', label='ERA5')
# VA
lwdn = va.lw_dn_srf.loc[t1:t2]
ax.plot(time_va, lwdn, color='black', linestyle='none', marker='^', label='VarAnal')
# FNL
time_fnl = np.arange(0, 23.9, 3.)
lwdn = fnl_sfc_ena.dlwrf.loc[t1:t2]
ax.plot(time_fnl, lwdn, color='black', linestyle='none', marker='*', markersize=10, label='FNL')
# MERRA2
lwdn = merra2_cld_ena.LWGAB.loc[t1:t2]
ax.plot(time_merra2, lwdn, color='black', linestyle='none', marker='o', markersize=5, label='MERRA2')
plt.xlabel('Time (UTC)')
plt.xlim(0, 24)
plt.xticks((0, 6, 12, 18))
# plt.ylim(0, 1.05)
plt.ylabel(r'LWDN ($W/m^2$)')
plt.title(datestring)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
datestring = '2018-02-16'
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# 1200 UTC
timestring = '12:00'
tstamp = f'{datestring}T{timestring}:00'
col = 'red'
# ERA5
center = 'ERA5'
marker_args = {'marker':'+'}
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, tstamp)
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
qc = era5_l_ena.clwc.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = era5_l_ena.cc.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# FNL
center = 'FNL'
marker_args = {'marker':'*'}
z = fnl_l_ena.gh.loc[tstamp]
lim = z < 5000.
zlim = np.compress(lim, z)
qc = fnl_l_ena.clwmr.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# MERRA2
center = 'MERRA2'
marker_args = {'marker':'o', 'markersize':5}
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, tstamp)
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
qc = merra2_l_ena.QL.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = merra2_l_ena.CLOUD.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cbh = cbh + 30.
cbh = np.nanmean(np.where(arscl.cloud_base_best_estimate.loc[f'{datestring}T11:30:00':f'{datestring}T12:30:00'] > 0.0, arscl.cloud_base_best_estimate.loc[f'{datestring}T11:30:00':f'{datestring}T12:30:00'], np.nan))
ax1.axhline(cbh*1.0e-3, color='red')
ax2.axhline(cbh*1.0e-3, color='red')
# 1800 UTC
timestring = '18:00'
tstamp = f'{datestring}T{timestring}:00'
col = 'blue'
# ERA5
center = 'ERA5'
marker_args = {'marker':'+'}
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, tstamp)
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
qc = era5_l_ena.clwc.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = era5_l_ena.cc.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# FNL
center = 'FNL'
marker_args = {'marker':'*'}
z = fnl_l_ena.gh.loc[tstamp]
lim = z < 5000.
zlim = np.compress(lim, z)
qc = fnl_l_ena.clwmr.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
# MERRA2
center = 'MERRA2'
marker_args = {'marker':'o', 'markersize':5}
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, tstamp)
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
qc = merra2_l_ena.QL.loc[tstamp]
qc = np.compress(lim, qc)*1.0e3
ax1.plot(qc, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cf = merra2_l_ena.CLOUD.loc[tstamp]
cf = np.compress(lim, cf)
ax2.plot(cf, zlim*1.0e-3, color=col, linestyle='none', label=f'{center} {timestring}', **marker_args)
cbh = np.nanmean(np.where(arscl.cloud_base_best_estimate.loc[f'{datestring}T17:30:00':f'{datestring}T18:30:00'] > 0.0, arscl.cloud_base_best_estimate.loc[f'{datestring}T17:30:00':f'{datestring}T18:30:00'], np.nan))
cbh = cbh + 30.
ax1.axhline(cbh*1.0e-3, color='blue')
ax2.axhline(cbh*1.0e-3, color='blue')
ax1.set_ylim(0, 5)
ax1.set_ylabel('Altitude (km)')
ax1.set_xlabel(r'$r_c$ (g/kg)')
ax1.set_title(datestring)
ax1.legend(fontsize=12)
ax2.set_ylim(0, 5)
ax2.set_ylabel('')
ax2.set_xlabel('Cloud Fraction')
ax2.set_title(datestring)
plt.show()